# Practice 6 & 7
#### In the following code , only two of the function usages are needed. One is is_valid(self, state), which is to determine if the current state is legal; the other is is_satisfy(self, state), which is to determine if the current board meets the win condition. 
#### The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.
#### In the first line of the code you can see N = 5, this is the size of the test.
#### Then in the test_block that follows, render indicates whether to use a graphical interface representation, and then method indicates which bts are used(bts or improving_bts).

### Practice 6: You should complete the function bts(problem). 
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. And you can add any function you want. 
### Practice 7: You should complete the function improving_bts(problem). 
You can select one or more methods of the three methods below(Minimum Remaining Value, Least constraining value, Forward checking), but you should have a good performance **when N = 16 without GUI**. 

In [5]:
# Look The Note Below!!!

import numpy as np
import time

N = 30 # You shoule change the test size here !!!

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

# BTS: Backtracking search (Practice 6)

In [6]:
def bts(problem):
    # action_stack = [(0, -1)]  
    # while action_stack:
    #     i, j = action_stack.pop()  
    #     j += 1  
    #     while j < problem.n:
    #         if problem.is_valid(action_stack + [(i, j)]):  
    #             action_stack.append((i, j))  
    #             if len(action_stack) == problem.n:
    #                 problem.print_state(action_stack)
    #                 yield action_stack  
    #             else:
    #                 action_stack.append((i + 1, -1))
    #             break  
    #         j += 1  
    action_stack = []
    num = [0] * problem.n
    count = 0
    while not problem.is_satisfy(action_stack):
        stack = num[count]
        ok = False
        for x in range(problem.n):
            if x < stack:
                continue
            test = action_stack.copy()
            test.append((count, x))
            if not problem.is_valid(test):
                continue
            action_stack.append((count, x))
            num[count] = x + 1
            count += 1
            ok = True
            break
        if not ok:
            num[count] = 0
            count -= 1
            action_stack.pop(-1)
            num[count] += 1
        yield action_stack

# Improving BTS To DO (Practice 7)
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [7]:
def improving_bts(problem):
    action_stack = []
    num = [-1] * problem.n

    def get_available_positions(action):
        available_positions = [[True] * problem.n for _ in range(problem.n)]
        for row, col in action:
            for i in range(problem.n):
                available_positions[row][i] = False
                available_positions[i][col] = False
            for i in range(problem.n):
                if 0 <= row + i < problem.n and 0 <= col + i < problem.n:
                    available_positions[row + i][col + i] = False
                if 0 <= row - i < problem.n and 0 <= col + i < problem.n:
                    available_positions[row - i][col + i] = False
                if 0 <= row + i < problem.n and 0 <= col - i < problem.n:
                    available_positions[row + i][col - i] = False
                if 0 <= row - i < problem.n and 0 <= col - i < problem.n:
                    available_positions[row - i][col - i] = False
        return available_positions
                    
    def forward_checking(action_stack):
        available_positions = get_available_positions(action_stack)
        occupied = [False] * problem.n
        for row, _ in action_stack:
            occupied[row] = True
        for row in range(problem.n):
            if occupied[row]:
                continue
            act_num = 0
            for col in range(problem.n):
                if available_positions[row][col]:
                    act_num += 1
            if act_num == 0:
                return False
        return True
    
    def get_minimal_remaining_value_index(action_stack):
        available_positions = get_available_positions(action_stack)
        occupied = [False] * problem.n
        for row, _ in action_stack:
            occupied[row] = True
        min_index = -1
        min_num = problem.n + 1
        for row in range(problem.n):
            if occupied[row]:
                continue
            act_num = 0
            for col in range(problem.n):
                if available_positions[row][col]:
                    act_num += 1
            if act_num < min_num:
                min_num = act_num
                min_index = row
        return min_index
                
    while not problem.is_satisfy(action_stack):
        min_index = get_minimal_remaining_value_index(action_stack)
        # print('choose row:', min_index)
        find = False
        for col in range(problem.n):
            if col <= num[min_index]:
                continue
            test = action_stack.copy()
            test.append((min_index, col))
            if not problem.is_valid(test):
                # print('position:', min_index, col, 'is invalid')
                continue
            if not forward_checking(test):
                # print('position:', min_index, col, 'fail forward checking')
                continue
            num[min_index] = col
            action_stack.append((min_index, col))
            num[min_index] = col
            find = True
            break
        if not find:
            num[min_index] = -1
            action_stack.pop(-1)
        yield action_stack
    
    

In [8]:

# test_block
n = N # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
render = False # here to select GUI or not
method = improving_bts  # here to change your method; bts or improving_bts
p = Problem(n)
if render:
    import pygame
    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = method(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen.png')
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    cnt = 0
    for actions in method(p):
        cnt += 1
    p.print_state(actions)
    print(f'Found a solution in {cnt} steps')
    print(time.time() - start_time)


_____________________________________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·